In [1]:
import sys
sys.path.append('/lmh_data/work/sclab/train/')

import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
from torch.autograd import Variable
from captum.attr import IntegratedGradients

from dataset import Dataset
from net import define_network

In [2]:
torch.manual_seed(123)
np.random.seed(123)

In [3]:
def evaluate(eval_file, model_file, gene_name):
    eval_set = Dataset(eval_file, gene_name, is_train=False)
    data_loader = data.DataLoader(eval_set, batch_size=1, shuffle=False)

    input_size, output_size = tuple(eval_set[0][0].shape), 78
    patch_size = tuple([int(i / 8) for i in input_size])
    model = torch.nn.DataParallel(define_network(input_size, patch_size, output_size))
    model.load_state_dict(torch.load(model_file))
    model.cuda()
    model.eval()
    ig = IntegratedGradients(model)

    _datas = np.load(eval_file, allow_pickle=True)
    important_genes = dict(ASC=dict(), Endo=dict(), OPC=dict(), ODC=dict())
    for index, batch in enumerate(data_loader, 1):
        if _datas[index-1]['cell_type'] != 'OPC':
            continue
        _important_genes = important_genes[_datas[index-1]['cell_type']]
        input = Variable(batch[0]).cuda().unsqueeze(1)
#         target = Variable(batch[1]).cuda().unsqueeze(1) / 10
        target = model(input)
        
        attributions = torch.zeros(input.reshape(-1).shape).cuda()
        for i in range(target.shape[-1]):
            attributions += ig.attribute(input, target=(0, i))[0, 0].reshape(-1)
        attributions = attributions.cpu().detach().numpy()
        for i in range(len(attributions)):
            if i not in _important_genes.keys():
                _important_genes[i] = 0
            _important_genes[i] += attributions[i]
        
    return important_genes

important_genes = evaluate(
    '/lmh_data/data/sclab/sclab/AD/eval_dataset.npy',
    '/lmh_data/data/sclab/sclab/AD/tmp/VCAN/model_epoch_8.pth', 'VCAN')
important_genes

{'ASC': {},
 'Endo': {},
 'OPC': {0: -0.037259137563523836,
  1: -0.004840989480726421,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: -0.040690075838938355,
  6: 0.0,
  7: -0.009027342079207301,
  8: 0.0,
  9: 0.0,
  10: -0.024896779446862638,
  11: -0.03968928148970008,
  12: -0.005921403644606471,
  13: 0.0,
  14: 0.008554132680728799,
  15: -0.0026379521004855633,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: -0.010381766012869775,
  20: -0.10447679739445448,
  21: 0.0,
  22: 0.0025839269437710755,
  23: 0.0,
  24: 0.0,
  25: 0.0,
  26: 0.05353290063794702,
  27: 0.0,
  28: 0.0,
  29: 0.0,
  30: -0.10367873264476657,
  31: -0.06534323538653553,
  32: 0.0,
  33: 0.0003322294969620998,
  34: 0.0,
  35: -0.006909220363013446,
  36: 0.0021139572781976312,
  37: 0.0,
  38: 0.0,
  39: -0.05748670201865025,
  40: 0.0,
  41: 0.0,
  42: 0.009623652789741755,
  43: -0.012334012892097235,
  44: -0.07698950311169028,
  45: 0.0,
  46: -0.07637348934076726,
  47: -0.02886008913628757,
  48: -0.005351327883545309,


In [5]:
scRNA_head = np.load('/lmh_data/data/sclab/sclab/AD/eval_dataset.npy', allow_pickle=True)[0]['scRNA_head']

def find_special_genes(important_genes, cell_type):
    genes = pd.DataFrame(
        sorted(important_genes[cell_type].items(), key=lambda x: x[1], reverse=True), columns=['index', 'value']
    ).set_index('index')
    genes = genes.sort_values(by=['value'], ascending=False)
    return genes

genes = find_special_genes(important_genes, 'OPC')
genes['name'] = scRNA_head[list(genes.index)]
genes.to_csv('tmp/related_genes_VCAN.csv')
genes

,value,name
index,,
9870,22.000219,TMEM108
16700,8.659179,KAZN
11461,7.840885,FRMD4A
16509,6.182408,LRRC4C
1486,5.726314,MDGA2
...,...,...
9856,-4.995712,TNS3
6320,-5.348876,SMOC1
9859,-5.572204,MAML2
